# Работа с БД SQLite

In [266]:
import sqlite3
from prettytable import from_db_cursor

In [267]:
import os 

# временный каталог
TEMP_DIR = 'temp'

if not os.path.exists(TEMP_DIR):
    os.mkdir(TEMP_DIR)

# Пример использования    
os.path.join(TEMP_DIR, "temp_SQLite.db")

'temp\\temp_SQLite.db'

In [268]:
db_name = os.path.join(TEMP_DIR, "temp_SQLite.db")

if os.path.isfile(db_name):
    os.remove(db_name)
    print(f"Файл [{db_name}] удален")

PermissionError: [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: 'temp\\temp_SQLite.db'

In [ ]:
db = sqlite3.connect(db_name)
cur = db.cursor()

In [ ]:
#cur.execute("CREATE TABLE cities (name)")
#cur.execute("CREATE TABLE cities (name varchar(100))")
cur.execute("CREATE TABLE cities (id integer primary key autoincrement not null, name varchar(100) unique)")
db.commit()

In [ ]:
cur.execute("SELECT * FROM cities")
from_db_cursor(cur)

id,name


In [ ]:
cur.execute("PRAGMA table_info('cities')")
from_db_cursor(cur)

cid,name,type,notnull,dflt_value,pk
0,id,integer,1,None,1
1,name,varchar(100),0,None,0


In [ ]:
cur.execute("PRAGMA table_info('cities')")
t = from_db_cursor(cur)
print(t)

+-----+------+--------------+---------+------------+----+
| cid | name |     type     | notnull | dflt_value | pk |
+-----+------+--------------+---------+------------+----+
|  0  |  id  |   integer    |    1    |    None    | 1  |
|  1  | name | varchar(100) |    0    |    None    | 0  |
+-----+------+--------------+---------+------------+----+


In [ ]:
cities = 'Москва, Воронеж, Калуга, Тула'.split(', ')
cities

['Москва', 'Воронеж', 'Калуга', 'Тула']

In [ ]:
cities_data = [(i,) for i in cities]
cities_data

[('Москва',), ('Воронеж',), ('Калуга',), ('Тула',)]

In [ ]:
cur.execute("INSERT INTO cities (name) VALUES ('Ялта')")
cur.execute("SELECT * FROM cities")
from_db_cursor(cur)

id,name
1,Ялта


In [ ]:
cur.executemany("INSERT INTO cities (name) VALUES (?)",cities_data)

cur.execute("SELECT * FROM cities")
from_db_cursor(cur)

id,name
1,Ялта
2,Москва
3,Воронеж
4,Калуга
5,Тула


In [ ]:
cur.execute("SELECT * FROM cities")
print(*cur)

(1, 'Ялта') (2, 'Москва') (3, 'Воронеж') (4, 'Калуга') (5, 'Тула')


In [ ]:
for line in cur.execute("SELECT * FROM cities"):
    print(line)

(1, 'Ялта')
(2, 'Москва')
(3, 'Воронеж')
(4, 'Калуга')
(5, 'Тула')


In [ ]:
cur.execute("SELECT * FROM cities ORDER BY name")
from_db_cursor(cur)

id,name
3,Воронеж
4,Калуга
2,Москва
5,Тула
1,Ялта


In [ ]:
# стиль qmark - использование одного или нескольких вопросиков
sql = "INSERT INTO cities (name) VALUES (?)"

cur.execute(sql,("London",))
cur.execute(sql,("Berlin",))

cur.execute("SELECT * FROM cities")
from_db_cursor(cur)

id,name
1,Ялта
2,Москва
3,Воронеж
4,Калуга
5,Тула
6,London
7,Berlin


In [ ]:
# стиль именованный - использование одного или нескольких ':<название>'
sql = "INSERT INTO cities (name) VALUES (:city)"

cur.execute(sql,{"city": "Belgrade"})

cur.execute("SELECT * FROM cities")
from_db_cursor(cur)

id,name
1,Ялта
2,Москва
3,Воронеж
4,Калуга
5,Тула
6,London
7,Berlin
8,Belgrade


In [ ]:
sql = "DELETE FROM cities where id = ?"

cur.execute(sql, [6])

from_db_cursor(cur.execute("SELECT * FROM cities"))

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 1, and there are 2 supplied.

In [ ]:
db.close()